### Project 1: Unit Testing

TimeZone Class

In [1]:
import numbers
import itertools
from datetime import timedelta
from datetime import datetime

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0: 
            raise ValueError('TimeZone name cannot be empty')

        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minutes offset must be an integer")

        if abs(offset_minutes) > 59 :
            raise ValueError("Minutes offset must be [-59, 59].")

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError("Offset must be between -12.00 and 14.00")

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return(isinstance(other, TimeZone) and
               self.name == other.name and
               self._offest_hours == other._offset_hours and
               self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return(f"TimeZone(name='{self.name}', "
               f"offest_hours={self._offset_hours}, "
               f"offset_minutes={self._offset_minutes})")


In [2]:
class Account:
    transaction_counter = itertools.count(100)

    _interset_rate = 0.5 # percent

    _transaction_codes = {
        "deposit": "D",
        "withdraw": "W",
        "interest": "I",
        "rejected": "X",
    }

    def __init__(self, account_number, first_name, last_name, timezone=None, 
                initial_balance=0):
        self._account_number = account_number
        self._first_name = first_name
        self._last_name = last_name
        # print("Account is created!")

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone

        self._balance = float(initial_balance)

    @property
    def account_number(self):
        return self._account_number

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("First name cannot be empty!")
        # self._first_name = value
        self._first_name = self.validate_and_set_name('_first_name', value, "First Name")

    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        # if len(str(value).strip()) == 0:
        #     raise ValueError("Last name cannot be empty!")
        # self._last_name = value
        self._last_name = self.validate_and_set_name("_last_name", value, "Last Name")

    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def balance(self):
        return round(self._balance, 2)
    
    @property
    def timezone(self):
        return self._timezone

    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time must be valid')
        self._timezone = value

    @classmethod
    def get_iterest_rate(cls):
        return cls._interset_rate

    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError("Interest rate must be a real number")

        if value < 0:
            raise ValueError("must be above zero")

        cls._interset_rate = value

    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError("Value must be a real number")

        if min_value is not None and value < min_value:
            raise ValueError(f"Value must be at least {min_value}")

        return value
    def generate_confirmation_code(self, transaction_code):
        dt_str = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        return f"{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}"
        
    # @staticmethod
    def validate_and_set_name(self, attr_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f"{field_title} cannot be empty!")
        # return str(value).strip()
        setattr(self, attr_name, value)

    # def make_transaction(self):
    #     return self.generate_confirmation_code("tummy")

    @staticmethod
    def parse_confirmation_code(confiramtion_code, preferred_time_zone=None):
        parts = confiramtion_code.split("-")
        if len(parts) != 4:
            raise ValueError("Invalid confirmation code")

        transaction_code, account_number, raw_dt_utc, transaction_id = parts

        try:
            dt_utc = datetime.strptime(raw_dt_utc, "%Y%m%d%H%M%S")

        except ValueError as ex:
            raise ValueError("Invalid transaction datetime") from ex

        if preferred_time_zone is None:
            preferred_time_zone = TimeZone("UTC", 0, 0)

        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError("Invalid TimeZone provided.")

        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str  = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"

        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)

    def deposit(self, value):
        
        value = Account.validate_real_number(value, 0.01)
            
        transaction_code = self._transaction_codes['deposit']

        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += value

        return conf_code

    def withdraw(self, value):
        # TODO: refactor to use common validation and in deposit method
        value = Account.validate_real_number(value, 0.01)
        accepted = False
        if self.balance - value < 0:
            transaction_code = self._transaction_codes['rejected']

        else:
            accepted = True
            transaction_code = self._transaction_codes['withdraw']

        conf_code = self.generate_confirmation_code(transaction_code)

        if accepted:
            self._balance -= value

        return conf_code

    def pay_interest(self):
        interest = self.balance * self.get_iterest_rate() / 100
        conf_code = self.generate_confirmation_code(self._transaction_codes['interest'])
        self._balance += interest
        return conf_code

In [3]:
a = Account("A100", "Rita", "Idle", initial_balance=120, timezone=TimeZone("MST", -7, 0))
print(a.balance)
print(a.deposit(100.02))
Account.set_interest_rate(1.0)
print(a.get_iterest_rate())
print(a.pay_interest())
print(a.balance)
# assert

120.0
D-A100-20230703014707-100
1.0
I-A100-20230703014707-101
222.22


In [32]:
try:
    a.deposit(-100)
except ValueError as ex:
    print(ex)

Value must be at least 0.01


In [28]:
a.balance

220.0

In [29]:
a.withdraw(400)

'X-A100-20230703005609-101'

In [27]:
try:
    Account.parse_confirmation_code("tummy-A100-20231629190256-101")
except ValueError as ex:
    print(ex.__cause__)
    print(ex)

unconverted data remains: 0256
Invalid transaction datetime


In [15]:
a.make_transaction()

'tummy-A100-20230629182234-100'

In [16]:
from collections import namedtuple

Confirmation = namedtuple("Confiramtion", "account_number transaction_code transaction_id time_utc time")

In [10]:
def generate_confirmation_code(account_number, transaction_id, transaction_code):
    dt_str = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    return f"{transaction_code}-{account_number}-{dt_str}-{transaction_id}"

In [11]:
generate_confirmation_code(1234, 10023, "X")

'X-1234-20230629174424-10023'

In [5]:
Account.get_iterest_rate()

0.5

In [7]:
Account.set_interest_rate(11)
Account.get_iterest_rate()

11

In [10]:
try:
    a = Account(123, "July", "Clark", "-7.00")
except ValueError as ex:
    print(ex)

Time must be valid


In [13]:
b = Account('23211', "Bob", "Dilan")


In [14]:
b.interset_rate

0.5

In [8]:
b.full_name

'Bob Dilan'

In [3]:
tz1 = TimeZone("ABC", -2, -15)
tz1.name

'ABC'

In [4]:
from datetime import datetime

dt = datetime.utcnow()
print(dt)

2023-05-10 01:09:41.742426


In [5]:
print(dt + tz1.offset)

2023-05-09 22:54:41.742426


In [6]:
try:
    tz = TimeZone("", 1, 10)
except ValueError as ex:
    print(ex)

TimeZone name cannot be empty


In [8]:
class TransactionID:
    def __init__(self, start_id):
        self._start_id = start_id

    def next(self):
        self._start_id += 1
        return self._start_id

In [9]:
class Account:
    transaction_counter = TransactionID(100)

    def make_transaction(self):
        new_trans_id = Account.transaction_counter.next()
        return new_trans_id

In [14]:
ac1 = Account()
ac2 = Account()

In [15]:
print(ac1.make_transaction())
print(ac2.make_transaction())

101
102


In [12]:
def transaction_ids(start_id):
    while True:
        start_id += 1
        yield start_id

In [13]:
class Account:
    transaction_counter = transaction_ids(100)

    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id

Using module `itertools`

In [17]:
import itertools

class Account:
    transaction_counter = itertools.count(100)

    def make_transaction(self):
        new_trans_id = next(Account.transaction_counter)
        return new_trans_id

In [18]:
ac1 = Account()
ac2 = Account()

print(ac1.make_transaction())
print(ac2.make_transaction())
print(ac2.make_transaction())

100
101
102
